In [1]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017")
db = client["spotify_db"]

artists_col = db.spotify_artists
tracks_col = db.spotify_artist_tracks
combined_col = db.spotify_artist_top10

In [4]:
artists_df = pd.read_json("Spotify/spotify_artist_info_20251201.json")

artists_col.delete_many({})
artists_col.insert_many(artists_df.to_dict(orient="records"))

InsertManyResult([ObjectId('6931c1e86feef089ae7cbc10'), ObjectId('6931c1e86feef089ae7cbc11'), ObjectId('6931c1e86feef089ae7cbc12'), ObjectId('6931c1e86feef089ae7cbc13'), ObjectId('6931c1e86feef089ae7cbc14'), ObjectId('6931c1e86feef089ae7cbc15'), ObjectId('6931c1e86feef089ae7cbc16'), ObjectId('6931c1e86feef089ae7cbc17'), ObjectId('6931c1e86feef089ae7cbc18'), ObjectId('6931c1e86feef089ae7cbc19'), ObjectId('6931c1e86feef089ae7cbc1a'), ObjectId('6931c1e86feef089ae7cbc1b'), ObjectId('6931c1e86feef089ae7cbc1c'), ObjectId('6931c1e86feef089ae7cbc1d'), ObjectId('6931c1e86feef089ae7cbc1e'), ObjectId('6931c1e86feef089ae7cbc1f'), ObjectId('6931c1e86feef089ae7cbc20'), ObjectId('6931c1e86feef089ae7cbc21'), ObjectId('6931c1e86feef089ae7cbc22'), ObjectId('6931c1e86feef089ae7cbc23'), ObjectId('6931c1e86feef089ae7cbc24'), ObjectId('6931c1e86feef089ae7cbc25'), ObjectId('6931c1e86feef089ae7cbc26'), ObjectId('6931c1e86feef089ae7cbc27'), ObjectId('6931c1e86feef089ae7cbc28'), ObjectId('6931c1e86feef089ae7cbc

In [6]:
tracks_df = pd.read_json("Spotify/spotify_artist_tracks_20251201.json")
tracks_col.delete_many({})
tracks_col.insert_many(tracks_df.to_dict(orient="records"))

InsertManyResult([ObjectId('6931c2156feef089ae7cc0fd'), ObjectId('6931c2156feef089ae7cc0fe'), ObjectId('6931c2156feef089ae7cc0ff'), ObjectId('6931c2156feef089ae7cc100'), ObjectId('6931c2156feef089ae7cc101'), ObjectId('6931c2156feef089ae7cc102'), ObjectId('6931c2156feef089ae7cc103'), ObjectId('6931c2156feef089ae7cc104'), ObjectId('6931c2156feef089ae7cc105'), ObjectId('6931c2156feef089ae7cc106'), ObjectId('6931c2156feef089ae7cc107'), ObjectId('6931c2156feef089ae7cc108'), ObjectId('6931c2156feef089ae7cc109'), ObjectId('6931c2156feef089ae7cc10a'), ObjectId('6931c2156feef089ae7cc10b'), ObjectId('6931c2156feef089ae7cc10c'), ObjectId('6931c2156feef089ae7cc10d'), ObjectId('6931c2156feef089ae7cc10e'), ObjectId('6931c2156feef089ae7cc10f'), ObjectId('6931c2156feef089ae7cc110'), ObjectId('6931c2156feef089ae7cc111'), ObjectId('6931c2156feef089ae7cc112'), ObjectId('6931c2156feef089ae7cc113'), ObjectId('6931c2156feef089ae7cc114'), ObjectId('6931c2156feef089ae7cc115'), ObjectId('6931c2156feef089ae7cc1

In [7]:
pipeline = [

    # join tracks with artists
    {
        "$lookup": {
            "from": "spotify_artist_tracks",
            "localField": "artist_id",
            "foreignField": "artist_id",
            "as": "tracks"
        }
    },

    # sort tracks by popularity DESC and keep only 10
    {
        "$addFields": {
            "tracks": {
                "$slice": [
                    { "$sortArray": { "input": "$tracks", "sortBy": { "popularity": -1 } } },
                    10
                ]
            }
        }
    },

    # final shape
    {
        "$project": {
            "_id": 0,
            "artist_id": 1,
            "name": 1,
            "genres": 1,
            "followers": 1,
            "popularity": 1,
            "tracks.track_id": 1,
            "tracks.track_name": 1,
            "tracks.popularity": 1,
            "tracks.album_name": 1,
            "tracks.release_date": 1
        }
    }
]


In [8]:
combined = list(artists_col.aggregate(pipeline))

combined_col.delete_many({})
combined_col.insert_many(combined)

print("Combined artist+top10 tracks stored successfully.")

Combined artist+top10 tracks stored successfully.


In [9]:
# mongoexport \
#   --uri="mongodb://localhost:27017" \
#   --db spotify_db \
#   --collection combined_col \
#   --out combined_col.json